In [2]:
import os

In [3]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

### Clone Github Repository  
https://github.com/VaibhavBomle/Llama-2-Project-on-CPU

In [4]:
%pwd

'c:\\GenerativeAI-Basics\\Project-Generative-AI\\source-code-analysis-project-GenAl\\research'

In [3]:
!mkdir test_repo

In [5]:
repo_path = "test_repo/"
Repo.clone_from("https://github.com/VaibhavBomle/Llama-2-Project-on-CPU",to_path=repo_path)

<git.repo.base.Repo 'c:\\GenerativeAI-Basics\\Project-Generative-AI\\source-code-analysis-project-GenAl\\research\\test_repo\\.git'>

In [8]:
repo_path = "test_repo/"

loader = GenericLoader.from_filesystem(repo_path+'/src',
                                       glob="**/*",
                                       suffixes=[".py"],
                                       parser=LanguageParser(language=Language.PYTHON,parser_threshold=300))

In [9]:
documents  = loader.load()

In [10]:
documents

[Document(page_content='DEFAULT_SYSTEM_PROMPT="""\\\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. \nYour answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. \nPlease ensure that your responses are socially unbiased and positive in nature.\nIf a question does not make any sense, or is not factually coherent, explain why instead of \nanswering something not correct. If you don\'t know the answer to a question,\nplease don\'t share false information."""\n\n\n\n#CUSTOM_SYSTEM_PROMPT="You are an advanced assistant that provides translation from English to Hindi"\nCUSTOM_SYSTEM_PROMPT="You are an advanced assistant that provides summarization given any book name"\n\n\n\n\ntemplate="""Use the following pieces of information to answer the user\'s question.\nIf you dont know the answer just say you know, don\'t try to make up an answer.\n\nContext:{context}\nQuestion:{question}\

### Chunkings

In [13]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON,
                                                                 chunk_size = 2000,
                                                                 chunk_overlap=200)

In [14]:

texts = documents_splitter.split_documents(documents)

In [15]:
len(texts)


3

### Embadding model

In [31]:
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"


In [29]:
embeddings=OpenAIEmbeddings(disallowed_special=())


### Knowledge base (vector DB)

In [ ]:

vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./data')
vectordb.persist()

### LLM Wrapper


In [ ]:
# llm = ChatOpenAI(model_name="gpt-4")
llm = ChatOpenAI()

In [ ]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)


In [ ]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":3}), memory=memory)


### Q & A

In [ ]:
question = "what is PromptTemplate?"


In [ ]:

result = qa(question)
print(result['answer'])